# Querying options

It is possible to simply run a query with 
- either a vectorized version of some input supplied when running the query,
- or an object from our storage that results in the most relevant items to the said object.

In [1]:
%pip install superlinked==15.1.0

In [2]:
import pandas as pd
from superlinked import framework as sl

pd.set_option("display.max_colwidth", 100)

In [3]:
class Paragraph(sl.Schema):
    id: sl.IdField
    body: sl.String


paragraph = Paragraph()

body_space = sl.TextSimilaritySpace(
    text=paragraph.body, model="sentence-transformers/all-mpnet-base-v2"
)
paragraph_index = sl.Index(body_space)

Now let's add some data to our space and fire up a running executor

In [4]:
source: sl.InMemorySource = sl.InMemorySource(paragraph)
executor = sl.InMemoryExecutor(sources=[source], indices=[paragraph_index])
app = executor.run()

In [5]:
source.put(
    [
        {"id": "paragraph-1", "body": "Glorious animals live in the wilderness."},
        {
            "id": "paragraph-2",
            "body": "Growing computation power enables advancements in AI.",
        },
        {
            "id": "paragraph-3",
            "body": "The flora and fauna of a specific habitat highly depend on the weather.",
        },
    ]
)

## Using the .similar clause

Makes us able to supply query input unrelated to the stored vectors.

In [6]:
# we are creating a Param to reuse the query.
# For more info check the `dynamic_parameters.ipynb` feature notebook in this same folder.
similar_query = (
    sl.Query(paragraph_index)
    .find(paragraph)
    .similar(body_space, sl.Param("similar_input"))
)

In [7]:
similar_result_weather = app.query(similar_query, similar_input="rainfall")
similar_result_weather.to_pandas()

,body,id,similarity_score
0,The flora and fauna of a specific habitat highly depend on the weather.,paragraph-3,0.337601
1,Glorious animals live in the wilderness.,paragraph-1,0.094036
2,Growing computation power enables advancements in AI.,paragraph-2,0.044686


In [8]:
similar_result_it = app.query(similar_query, similar_input="progress in AI")
similar_result_it.to_pandas()

,body,id,similarity_score
0,Growing computation power enables advancements in AI.,paragraph-2,0.598643
1,Glorious animals live in the wilderness.,paragraph-1,0.007107
2,The flora and fauna of a specific habitat highly depend on the weather.,paragraph-3,-0.121885


## Using the .with_vector clause

Provides the opportunity to search with the vector of an object in our database. This is useful for example for recommending items for a user based on it's vector.

In [9]:
with_vector_query = (
    sl.Query(paragraph_index).find(paragraph).with_vector(paragraph, "paragraph-3")
)

In [10]:
with_vector_result = app.query(with_vector_query)
with_vector_result.to_pandas()

,body,id,similarity_score
0,The flora and fauna of a specific habitat highly depend on the weather.,paragraph-3,1.000000
1,Glorious animals live in the wilderness.,paragraph-1,0.310591
2,Growing computation power enables advancements in AI.,paragraph-2,-0.019059


The first result is the one we are searching with, the second is the more related one, and finally the less connected paragraph body comes.

## Combine them

With the use of weights, creating any combination of inputs is possible. Imagine a situation where we search for a term, `similar_input` in those paragraphs that are relevant to a specific paragraph, denoted by `paragraph_id`. It is possible to weight the input using `input_weight` `Param`, in the relation to the context the search takes place inside using `context_weight` `Param`. Note that the `Param` names are totally arbitrary, the clauses matter.

In [11]:
# we are using dynamic parameters again
combined_query = (
    sl.Query(paragraph_index)
    .find(paragraph)
    .similar(body_space, sl.Param("similar_input"), weight=sl.Param("input_weight"))
    .with_vector(paragraph, sl.Param("paragraph_id"), weight=sl.Param("context_weight"))
)

In [12]:
# equal weight
combined_result = app.query(
    combined_query,
    similar_input="progress in AI",
    paragraph_id="paragraph-3",
    input_weight=1,
    context_weight=1,
)
combined_result.to_pandas()

,body,id,similarity_score
0,The flora and fauna of a specific habitat highly depend on the weather.,paragraph-3,0.620921
1,Growing computation power enables advancements in AI.,paragraph-2,0.409828
2,Glorious animals live in the wilderness.,paragraph-1,0.224647


In [13]:
# upweight context - notice the score differences
combined_result_context = app.query(
    combined_query,
    similar_input="progress in AI",
    paragraph_id="paragraph-3",
    input_weight=0.25,
    context_weight=1,
)
combined_result_context.to_pandas()

,body,id,similarity_score
0,The flora and fauna of a specific habitat highly depend on the weather.,paragraph-3,0.785410
1,Growing computation power enables advancements in AI.,paragraph-2,0.518396
2,Glorious animals live in the wilderness.,paragraph-1,0.284158


In [14]:
# give more weight to query time input - the most relevant document changes
combined_result_input = app.query(
    combined_query,
    similar_input="progress in AI",
    paragraph_id="paragraph-3",
    input_weight=1,
    context_weight=0.25,
)
combined_result_input.to_pandas()

,body,id,similarity_score
0,Growing computation power enables advancements in AI.,paragraph-2,0.531181
1,The flora and fauna of a specific habitat highly depend on the weather.,paragraph-3,0.114589
2,Glorious animals live in the wilderness.,paragraph-1,0.075807


## Filter results based on score or position

In [15]:
# let's use combined query above with some preset params
params = {
    "similar_input": "progress in AI",
    "paragraph_id": "paragraph-3",
    "input_weight": 1,
    "context_weight": 0.25,
}

In [16]:
# return top 2 items
combined_query_limit_result = app.query(combined_query.limit(2), **params)
combined_query_limit_result.to_pandas()

,body,id,similarity_score
0,Growing computation power enables advancements in AI.,paragraph-2,0.531181
1,The flora and fauna of a specific habitat highly depend on the weather.,paragraph-3,0.114589


In [17]:
# return items with scores larger than 0.5
combined_query_radius_result = app.query(combined_query.radius(0.5), **params)
combined_query_radius_result.to_pandas()

,body,id,similarity_score
0,Growing computation power enables advancements in AI.,paragraph-2,0.531181
